# 단일 입력 PromptTemplate

In [1]:
from langchain.prompts import PromptTemplate

In [2]:
import langchain
langchain.__version__

'0.3.25'

In [3]:
# 템플릿 문자열: 프롬프트의 골격을 정의
# PromptTemplate 객체 생성: 인스턴스 변수 선언
# 포맷(fromat) 메서드 수행: PromptTemplate.format()
# LLM 호출
template_str = (
  "당신은 최고 수준의 마케팅 카피라이터입니다. \n"
  "아래 제품의 매력적인 홍보 문구를 100자 내외로 작성하십시오. \n\n"
  "제품명: {product_name}"
)

prompt = PromptTemplate.from_template(template_str)

input_str = input("제품명을 입력하세요")
formatted_prompt = prompt.format(product_name=input_str)
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터입니다. 
아래 제품의 매력적인 홍보 문구를 100자 내외로 작성하십시오. 

제품명: 


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

chain = prompt | llm

result = chain.invoke({"product_name": input_str})
print(result.content)

제품명을 알려주시면, 매력적인 홍보 문구를 작성해드리겠습니다.


# 다중 입력 PromptTemplate

In [5]:
multi_template_str = (
  "아래는 뉴스 기사 제목과 키워드입니다.\n"
  "이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하십시오.\n\n"
  "제목 : {title}\n"
  "키워드 : {keywords}\n"
)

summary_prompt = PromptTemplate(template=multi_template_str, input_variables=["title", "keywords"])

sample_title = "인공지능 기술의 발전과 미래"
sample_keywords = "머신러닝, 딥러닝, 산업 혁신"
formatted_summary_prompt = summary_prompt.format(title=sample_title, keywords=sample_keywords)
print(formatted_summary_prompt)

아래는 뉴스 기사 제목과 키워드입니다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하십시오.

제목 : 인공지능 기술의 발전과 미래
키워드 : 머신러닝, 딥러닝, 산업 혁신



In [6]:
result_summary = (summary_prompt | llm).invoke({
  "title": sample_title,
  "keywords": sample_keywords
})
print(result_summary.content)

인공지능 기술은 머신러닝과 딥러닝의 발전을 바탕으로 빠르게 진화하며 산업 전반에 혁신을 일으키고 있습니다. 이러한 기술들은 데이터 분석과 자동화 능력을 향상시켜 다양한 분야에서 효율성을 높이고 있으며, 앞으로도 지속적인 연구와 개발을 통해 미래 산업의 핵심 동력으로 자리매김할 것으로 기대됩니다.


# ChatPromptTemplate 를 이욯나 역할 기반 프롬프트

In [7]:
from langchain.prompts import ChatPromptTemplate

In [9]:
# 시나리오 : 파이썬 프로그램 전문가 역할 부여, 사용자가 파이썬 관련 질문에 친절히, 간경한 답변 하도록
sys_message = ("당신은 파이썬 분야의 뛰어난 전문가이자 조언자입니다."
               "사용자의 프로그래밍 질문에 친절하고 이해하기 쉽게 답변하십시오."
               )
user_message = "{question}" # 사용자 질문 -> 실행 시 입력값으로 채워진다.

chat_prompt = ChatPromptTemplate.from_messages([
  ("system", sys_message),
  ("user", user_message)
])

sample_question = "파이썬의 리스트 컴프리헨션(list comprehention)과 map 함수의 차이가 무엇이야?"

message = chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='당신은 파이썬 분야의 뛰어난 전문가이자 조언자입니다.사용자의 프로그래밍 질문에 친절하고 이해하기 쉽게 답변하십시오.', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬의 리스트 컴프리헨션(list comprehention)과 map 함수의 차이가 무엇이야?', additional_kwargs={}, response_metadata={})]


In [10]:
result = (chat_prompt | llm).invoke({"question", sample_question})
print(result.content)

안녕하세요! 파이썬에서 리스트 컴프리헨션과 `map()` 함수는 둘 다 기존 리스트나 이터러블을 기반으로 새 리스트를 만들 때 사용되지만, 각각의 특징과 차이점이 있습니다. 아래에서 쉽게 설명해 드릴게요.

### 1. 리스트 컴프리헨션 (List Comprehension)
- **문법이 간단하고 직관적임**
- `[표현식 for 변수 in 이터러블 if 조건]` 형태로 사용
- 조건문도 넣을 수 있어서 필터링도 가능
- 새 리스트를 바로 생성하며, 가독성이 좋음

**예제:**
```python
numbers = [1, 2, 3, 4, 5]
squares = [x * x for x in numbers]
# 결과: [1, 4, 9, 16, 25]
```

### 2. `map()` 함수
- **함수와 이터러블을 인자로 받음**
- `map(함수, 이터러블)` 형태로 사용
- 반환값은 `map` 객체이기 때문에, 리스트로 변환하려면 `list()`로 감싸야 함
- 함수형 프로그래밍 스타일에 가깝고, 여러 이터러블을 동시에 처리 가능

**예제:**
```python
numbers = [1, 2, 3, 4, 5]
squares = list(map(lambda x: x * x, numbers))
# 결과: [1, 4, 9, 16, 25]
```

---

### 차이점 정리
| 특징 | 리스트 컴프리헨션 | `map()` 함수 |
|---|---|---|
| 문법 | `[표현식 for 변수 in 이터러블 if 조건]` | `map(함수, 이터러블)` |
| 가독성 | 간단하고 직관적 | 함수형 스타일 선호 시 적합 |
| 조건문 | 가능 (`if`) | 조건문 직접 불가, 필터링은 `filter()`와 함께 사용 |
| 반환값 | 바로 리스트 | `map` 객체 → 리스트로 변환 필요 (`list()`) |
| 여러 이터러블 | 복잡 | 여러 이터러블 동시에 처리 가능 (`map(func, a, b, ...)`) |

---

### 결론
- **간단한 변환이나 필

# PartialPromptTemplate

In [11]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

In [12]:
role_system_template = "당신은 {role} 분야의 전문 지식인입니다. 가능한 한 자세히 답변해주세요."
system_prompt = SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template("{question}")

base_chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

partial_chat_prompt = base_chat_prompt.partial(role="주식 투자")

In [16]:
sample_question = "현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은 어떻습니까?"
message = partial_chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='당신은 주식 투자 분야의 전문 지식인입니다. 가능한 한 자세히 답변해주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은 어떻습니까?', additional_kwargs={}, response_metadata={})]


In [17]:
result = (partial_chat_prompt | llm).invoke({"question", sample_question})
print(result.content)

2025년 5월 현재 시장 상황을 고려할 때, 삼성전자 주식의 전망은 여러 요인에 따라 달라질 수 있습니다. 아래에 주요 분석 포인트를 상세히 설명드리겠습니다.

1. 글로벌 경제 및 시장 환경
- 글로벌 경기 회복 여부: 2025년 현재 세계 경제는 일부 지역에서 안정세를 보이고 있으나, 미국, 중국, 유럽 등 주요 경제권의 성장률과 금리 정책이 주가에 영향을 미칩니다.
- 인플레이션과 금리 정책: 미국 연준과 한국은행의 금리 정책이 지속적으로 변동한다면, 투자 심리와 자금 유입이 영향을 받습니다.
2. 반도체 산업 및 기술 트렌드
- 수요와 공급: 반도체 수요는 인공지능, 5G, 자율주행차, IoT 등 신기술 확산으로 지속적으로 증가하는 추세입니다. 삼성전자는 메모리 반도체와 시스템 반도체 모두 강력한 경쟁력을 갖추고 있어 수요 증가에 따른 실적 호조가 기대됩니다.
- 공급망 이슈: 글로벌 공급망 문제, 원자재 가격 변동, 생산설비 투자 상황이 주가에 영향을 미칠 수 있습니다.
3. 삼성전자의 재무 상태 및 전략
- 실적 전망: 2025년 1분기 이후 실적은 반도체 가격 회복, 스마트폰 판매량 증가, 디스플레이 사업의 성장 등에 힘입어 긍정적일 가능성이 높습니다.
- 투자 및 연구개발: 차세대 기술 개발, 친환경 사업 확대, 인수합병 전략 등은 장기 성장 동력을 강화할 수 있습니다.
4. 경쟁사 및 시장 점유율
- 글로벌 경쟁사와의 경쟁 구도: TSMC, 인텔, SK하이닉스 등과의 경쟁 상황이 주가에 영향을 미칩니다.
- 시장 점유율 변화: 스마트폰, 가전, 반도체 시장에서의 점유율 확대 또는 축소 여부도 중요한 변수입니다.
5. 정책 및 규제 환경
- 정부 정책: 한국 정부의 반도체 산업 지원 정책, 글로벌 무역 정책, 규제 강화 여부가 시장에 영향을 미칩니다.
- 환경·사회·지배구조(ESG): ESG 기준 강화로 인한 기업의 대응 전략도 투자자들의 관심사입니다.
**전망 요약**  
2025년 5월 현재, 삼성전자는 글로벌 반도체 시장의 성장과 기술 혁